In [ ]:
!python -m pip install langchain==0.1.0 openai==1.7.2 langchain-openai langchain-community==0.0.12 langchainhub==0.1.14
!python -m pip install langchain langchain-google-genai google-generativeai GoogleGenerativeAIEmbeddings



In [3]:
import os
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.prompts import (
    PromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    ChatPromptTemplate
)
from langchain.tools import tool


In [4]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)



True

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"),
                 model="gpt-4o-mini"
                )
response = llm.invoke("What is the capital of France?")
print(response.content)

The capital of France is Paris.


In [6]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")


In [7]:
# This part is for finding relevant PROJECTS based on their description
openai_project_index = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY")),
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name="openai_project_index",
    node_label="Project",
    text_node_properties=["name", "description", "complexity"],
    embedding_node_property="openai_embedding",
)

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector
import os

# Create embedding model
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Load Neo4j vector index using Gemini embeddings
gemini_project_index = Neo4jVector.from_existing_graph(
    embedding=embedding,
    url=os.getenv("NEO4J_URL"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name="gemini_project_index",
    node_label="Project",
    text_node_properties=["name", "description", "complexity"],
    embedding_node_property="gemini_embedding",
)


In [9]:
openai_result = openai_project_index.similarity_search("Content Marketing Software", k=2)
print(openai_result)


[Document(metadata={'startDate': neo4j.time.Date(2021, 3, 1), 'benefit': 9012225.0, 'endDate': neo4j.time.Date(2021, 6, 1), 'cost': 4018835.0, 'gemini_embedding': [0.0486227422952652, -0.06065458431839943, -0.038773227483034134, -0.05230659991502762, 0.06815632432699203, -0.004172889515757561, 0.0010621348628774285, -0.015995146706700325, 0.02045055851340294, 0.03755713999271393, 0.0027481524739414454, 0.03885732218623161, -0.004840642213821411, -0.031658731400966644, -0.013202202506363392, -0.03472987934947014, 0.00021636502060573548, 0.044656556099653244, 0.004589537624269724, 0.02098952792584896, 0.027651382610201836, -0.0147137101739645, 0.005257644224911928, -0.03875922039151192, -0.0021570581011474133, -0.020573539659380913, 0.028448117896914482, -0.05593819171190262, -0.001275060698390007, 0.012582208961248398, -0.014230377972126007, 0.014861313626170158, -0.04997812584042549, 0.06461513787508011, 0.007573023904114962, -0.009676593355834484, -0.03750740736722946, 0.0312901958823

In [10]:
gemini_result = gemini_project_index.similarity_search("Content Marketing Software", k=2)
print(openai_result)

[Document(metadata={'startDate': neo4j.time.Date(2021, 3, 1), 'benefit': 9012225.0, 'endDate': neo4j.time.Date(2021, 6, 1), 'cost': 4018835.0, 'gemini_embedding': [0.0486227422952652, -0.06065458431839943, -0.038773227483034134, -0.05230659991502762, 0.06815632432699203, -0.004172889515757561, 0.0010621348628774285, -0.015995146706700325, 0.02045055851340294, 0.03755713999271393, 0.0027481524739414454, 0.03885732218623161, -0.004840642213821411, -0.031658731400966644, -0.013202202506363392, -0.03472987934947014, 0.00021636502060573548, 0.044656556099653244, 0.004589537624269724, 0.02098952792584896, 0.027651382610201836, -0.0147137101739645, 0.005257644224911928, -0.03875922039151192, -0.0021570581011474133, -0.020573539659380913, 0.028448117896914482, -0.05593819171190262, -0.001275060698390007, 0.012582208961248398, -0.014230377972126007, 0.014861313626170158, -0.04997812584042549, 0.06461513787508011, 0.007573023904114962, -0.009676593355834484, -0.03750740736722946, 0.0312901958823

In [11]:
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain

In [12]:
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
)

/var/folders/90/h9hy_l096_jcf3041fnkklp00000gn/T/ipykernel_73597/3960208139.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [13]:
graph.refresh_schema()


In [14]:
cypher_generation_template = """
Task:
Generate a Cypher query for a Neo4j graph database.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
The schema is case-sensitive, so you must use the exact casing provided.

Schema:
{schema}

Note:
**Do not include _embedding data, which is a part of Project node as its parameter** as part of the output in your generated chyper query, when a question like "Provide me the detailed information" is asked.
Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything other than for you to construct a Cypher statement.
Do not run any queries that would add to or delete from the database.
Make sure to alias all nodes and relationships properly.
Use the `WITH` clause to pass aliased values between query parts for clarity.
If you need to divide numbers, make sure to filter the denominator to be non-zero.
Property values like names, statuses, and regions are case-sensitive. For example, 'In - Progress' is different from 'in-progress'.

Examples:
# How many projects are in the West region?
MATCH (p:Project)-[:LOCATED_IN]->(r:Region {{name: 'West'}})
RETURN count(p) AS numberOfProjects

# What is the total cost of all completed projects?
MATCH (p:Project)-[:HAS_STATUS]->(s:Status {{name: 'Completed'}})
RETURN sum(p.cost) AS totalCost

# List the names of projects managed by Brenda Chandler that are cancelled.
MATCH (p:Project)-[:MANAGED_BY]->(m:ProjectManager {{name: 'Brenda Chandler'}}),
      (p)-[:HAS_STATUS]->(s:Status {{name: 'Cancelled'}})
RETURN p.name AS projectName

# Which department is the 'Rhinestone' project assigned to?
MATCH (p:Project {{name: 'Rhinestone'}})-[:ASSIGNED_TO]->(d:Department)
RETURN d.name AS departmentName

The question is:
{question}
"""

In [15]:
cypher_generation_prompt = PromptTemplate(
    input_variables=["schema", "question"],
    template=cypher_generation_template
)

In [16]:
qa_generation_template_str = """
You are a project management assistant. Your task is to take the results from a Neo4j Cypher query and form a clear, human-readable response.
The provided information in the "Query Results" section is authoritative; you must never question it or use your internal knowledge to alter it.
Make the answer sound like a direct and helpful response to the user's question.

Query Results:
{context}

Question:
{question}

Instructions for your response:
- If the provided information is empty, respond by stating that you could not find an answer based on the available data. An empty result is indicated by: []
- If the information is not empty, you must provide a concise answer using only the provided results.
- Be careful with names that might contain punctuation, such as the department 'Admin & BI'. Ensure any list of names is presented clearly.
- Never state that you lack sufficient information if data is present in the query results. Always utilize the data provided.

Helpful Answer:
"""

In [17]:
qa_generation_prompt = PromptTemplate(
    input_variables=["context", "question"], 
    template=qa_generation_template_str
)

In [18]:
openai_cypher_chain = GraphCypherQAChain.from_llm(
    top_k=10,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
    validate_cypher=True,
    qa_prompt=qa_generation_prompt,
    cypher_prompt=cypher_generation_prompt,
    qa_llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    cypher_llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
)


question = "What is the cost of all projects in the south?"
response = openai_cypher_chain.invoke(question)
print(response, '\n\n', response.get("result"))




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Project)-[:LOCATED_IN]->(r:Region {name: 'South'})
RETURN sum(p.cost) AS totalCostInSouth
Full Context:
[{'totalCostInSouth': 87668114.0}]

> Finished chain.
{'query': 'What is the cost of all projects in the south?', 'result': 'The total cost of all projects in the south is $87,668,114.'} 

 The total cost of all projects in the south is $87,668,114.


In [19]:
gemini_cypher_chain = GraphCypherQAChain.from_llm(
    top_k=10,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
    validate_cypher=True,
    qa_prompt=qa_generation_prompt,
    cypher_prompt=cypher_generation_prompt,
    qa_llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0),
    cypher_llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0),
)

question = "Who was assigned and managed Skyhawks, what status and milestone is it currently in and what are the next steps and what is it about?"
response = gemini_cypher_chain.invoke(question)
print(response, '\n\n', response.get("result"))




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Project {name: 'Skyhawks'})
OPTIONAL MATCH (p)-[:ASSIGNED_TO]->(d:Department)
OPTIONAL MATCH (p)-[:MANAGED_BY]->(pm:ProjectManager)
OPTIONAL MATCH (p)-[:HAS_STATUS]->(s:Status)
OPTIONAL MATCH (p)-[:IN_PHASE]->(ph:Phase)
RETURN d.name AS assignedDepartment, pm.name AS projectManager, s.name AS currentStatus, ph.name AS currentPhase, p.description AS projectDescription
Full Context:
[{'assignedDepartment': 'eCommerce', 'projectManager': 'Jaylyn Mckenzie', 'currentStatus': 'Completed', 'currentPhase': 'Phase 1 - Explore', 'projectDescription': 'Is A Solution For Founders Who Want To Win At Content Marketing. We Use Proprietary Ai To Analyze The Competition For Your Topic, And To Help You Create Optimized Content Faster With State Of The Art Ai Generation.'}]

> Finished chain.
{'query': 'Who was assigned and managed Skyhawks, what status and milestone is it currently in and what are the next steps and what is it about

In [25]:
project_details_chat_template_str = """
Your job is to use the provided project data to answer 
questions about their status, description, startdate, enddate, etc. 
within the company. Use the following context to answer questions. 
Be as detailed as possible, but don't make up any information that's 
not from the context. If you don't know an answer, say you don't know.
{context}
"""

In [73]:
@tool("project-qa-tool", return_direct=False )
def project_qa_tool(query: str, is_openai:bool = False) -> str:
    """
    Use this tool for questions about the content, description, or concepts of specific projects.
    This is best
    Use this tool for questions about the content, description, or concepts of specific projects.
    This is best for open-ended or "fuzzy" questions like 'What are projects about content marketing?' or 'Tell me more about the Skyhawks project'.
    It is the primary tool for understanding what a project IS.
    The tool for open-ended or "fuzzy" questions like 'What are projects about content marketing?' or 'Tell me more about the Skyhawks project'.
    It is the primary tool for understanding what a project IS.
    """

    
    project_details_chat_system_prompt = SystemMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["context"],
            template=project_details_chat_template_str))
    
    human_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["question"],
            template="Can you provide details on: {question}?"))
    
    messages = [project_details_chat_template_str, human_prompt]
    
    qa_prompt = ChatPromptTemplate(
        messages=messages,
        input_variables=["context", "question"])

    if is_openai:
        llm = ChatOpenAI(model="gpt-3.5-turbo")
        retriever = openai_project_index.as_retriever()
    else:
        llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
        retriever = gemini_project_index.as_retriever()
    
    
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever = retriever,
        # ['stuff', 'map_reduce', 'refine', 'map_rerank']
        chain_type="stuff", )
    
    qa_chain.combine_documents_chain.llm_chain.prompt = qa_prompt
    response = qa_chain.invoke(query)
    return response.get("result")

In [74]:
@tool("general-qa-tool",  return_direct=False)
def general_qa_tool(query: str, is_openai:bool = False) -> str:
    """
    Use this tool for questions that require factual, quantitative, or specific data that is not in a project's description
    Use this tool for questions that require factual, quantitative, or specific data that is not in a project's description.
    This is the ONLY source of truth for project costs, benefits, statuses, dates, managers, departments, and regions.
    Use it for any questions involving calculations, counts, sums, lists, or direct lookups across the entire graph.
    For example: 'How many projects are in the West region?', 'What is the total cost of all completed projects?'.
    """

    if is_openai:
        response = openai_cypher_chain.invoke(query)
    else:
        response = gemini_cypher_chain.invoke(query)
    return response.get("result")

In [63]:
project_qa_tool.invoke("Provide me the detailed information of Skyhawks.")


'Name: Skyhawks\nDescription: Skyhawks is a solution for founders who want to win at content marketing. The company uses proprietary AI to analyze the competition for your topic and to help you create optimized content faster with state-of-the-art AI generation. The complexity of the project is high. \nStartDate: Not provided\nEndDate: Not provided'

In [64]:
general_qa_tool.invoke("Provide me the detailed information of Skyhawks.")




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Project {name: 'Skyhawks'})
RETURN p.name AS projectName, p.description AS projectDescription, p.startDate AS projectStartDate, p.endDate AS projectEndDate, p.completionPercentage AS projectCompletionPercentage, p.cost AS projectCost, p.benefit AS projectBenefit, p.complexity AS projectComplexity
Full Context:
[{'projectName': 'Skyhawks', 'projectDescription': 'Is A Solution For Founders Who Want To Win At Content Marketing. We Use Proprietary Ai To Analyze The Competition For Your Topic, And To Help You Create Optimized Content Faster With State Of The Art Ai Generation.', 'projectStartDate': neo4j.time.Date(2021, 3, 1), 'projectEndDate': neo4j.time.Date(2021, 6, 1), 'projectCompletionPercentage': 100, 'projectCost': 5785601.0, 'projectBenefit': 8630148.0, 'projectComplexity': 'High'}]

> Finished chain.


'The project "Skyhawks" is a high complexity project that started on March 1, 2021, and ended on June 1, 2021. It had a project completion percentage of 100% and incurred a cost of $5,785,601. The project generated a benefit of $8,630,148. The project aims to provide a solution for founders who want to excel in content marketing by using proprietary AI technology to analyze competition and create optimized content efficiently.'

## Creating OPEN AI & Gemini Agent

In [65]:
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_openai_tools_agent
)

In [66]:
agent_prompt = hub.pull("hwchase17/openai-functions-agent")


In [67]:
agent_prompt.pretty_print()


================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [68]:
tools = [project_qa_tool, general_qa_tool]


### Function to test OpenAI and Gemini

In [77]:
import pandas as pd
from tqdm import tqdm
import io
import contextlib
import re

# --- HELPER FUNCTION TO CLEAN THE OUTPUT ---
def clean_ansi_codes(text: str) -> str:
    """Removes ANSI escape codes (used for terminal colors) from a string."""
    ansi_escape = re.compile(r'\x1b\[[0-9;]*m')
    return ansi_escape.sub('', text)
# -------------------------------------------


def test_questions_with_cypher_extraction(agent_executor, is_openai =True):
    # --- 1. Define the list of test questions ---
    test_questions = [
        "What is the project cost and benefit for the 'Rhinestone' project?",
        "How many projects are currently 'In - Progress'?",
        "What is the total benefit of all projects managed by Brenda Chandler?",
        "Which department is the 'The Blue Bird' project assigned to?",
        "List the names of all 'Completed' projects in the 'North' region.",
        "Are there any projects about teaching people new things?",
        "Find projects related to improving business capital.",
        "Tell me about projects involving software solutions for marketing.",
        "What is the total cost of all projects that are about improving processes?",
        "Who is the project manager for the project described as a 'casual game'?",
        "In the East region, are there any completed projects related to e-commerce?",
        "What is the status of the project about 'ancestors'?",
        "This is not a real question and should probably fail.",
    ]

    # --- 2. Prepare a list to store the results ---
    results_list = []

    # --- 3. Set pandas display options for better viewing ---
    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.width', 1000)

    print("Starting batch processing of test questions...")

    # --- 4. Loop through questions, capture logs, extract cypher, and handle errors ---
    for question in tqdm(test_questions, desc="Processing Questions"):
        answer = "N/A"
        cypher_query = "N/A - Cypher not generated"
        log_output = ""

        try:
            log_capture_string = io.StringIO()
            with contextlib.redirect_stdout(log_capture_string):
                response = agent_executor.invoke({"input": question})

            log_output = log_capture_string.getvalue()

            cypher_match = re.search(
                r"Generated Cypher:\n(.*?)(?:\n\n|Full Context:)",
                log_output,
                re.DOTALL
            )

            if cypher_match:
                # Extract the raw query
                raw_cypher = cypher_match.group(1).strip()
                
                # *** NEW: Clean the ANSI codes from the extracted query ***
                cypher_query = clean_ansi_codes(raw_cypher)

            answer = response.get("output", "N/A - 'output' key not found in response.")

        except Exception as e:
            print(f"\nAn error occurred for question: '{question}'")
            print(f"   Error details: {e}\n")
            answer = f"Error: {e}"
        
        if is_openai:
            results_list.append({
                "Question": question,
                "OPENAI Generated Cypher": cypher_query,
                "OPENAI Final Answer": answer
            })
        else:
            results_list.append({
                "Question": question,
                "GEMINI Generated Cypher": cypher_query,
                "GEMINI Final Answer": answer
            })


    print("\nBatch processing complete.")

    # --- 5. Create the final DataFrame from the results ---
    results_df = pd.DataFrame(results_list)
    return results_df

## OPENAI Agent

In [ ]:

llm = ChatOpenAI(model="gpt-3.5-turbo")
agent = create_openai_tools_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
openai_result = test_questions_with_cypher_extraction(agent_executor, is_openai = True)
openai_result

Starting batch processing of test questions...


Processing Questions: 100%|██████████| 13/13 [00:42<00:00,  3.29s/it]


Batch processing complete.


,Question,OPENAI Generated Cypher,OPENAI Final Answer
0,What is the project cost and benefit for the 'Rhinestone' project?,"MATCH (p:Project {name: 'Rhinestone'})\nRETURN p.cost AS projectCost, p.benefit AS projectBenefit","The project cost for the 'Rhinestone' project is $3,648,615.00, and the project benefit is $8,443,980.00."
1,How many projects are currently 'In - Progress'?,MATCH (p:Project)-[:HAS_STATUS]->(s:Status {name: 'In - Progress'})\nRETURN count(p) AS numberOfProjects,There are currently 25 projects that are 'In - Progress'.
2,What is the total benefit of all projects managed by Brenda Chandler?,MATCH (p:Project)-[:MANAGED_BY]->(m:ProjectManager {name: 'Brenda Chandler'})\nRETURN sum(p.benefit) AS totalBenefit,"The total benefit of all projects managed by Brenda Chandler is $131,698,142."
3,Which department is the 'The Blue Bird' project assigned to?,MATCH (p:Project {name: 'The Blue Bird'})-[:ASSIGNED_TO]->(d:Department)\nRETURN d.name AS departmentName,The 'The Blue Bird' project is assigned to the Warehouse department.
4,List the names of all 'Completed' projects in the 'North' region.,"MATCH (p:Project)-[:HAS_STATUS]->(s:Status {name: 'Completed'}),\n (p)-[:LOCATED_IN]->(r:Region {name: 'North'})\nRETURN p.name AS projectName, p.description AS projectDescription, p.startDate AS projectStartDate, p.endDate AS projectEndDate, p.cost AS projectCost, p.benefit AS projectBenefit, p.completionPercentage AS projectCompletionPercentage","The completed projects in the North region are:\n1. Impact Training\n2. Gob Geeklords\n3. Leadership Insights\n4. Commission Kings\n5. Project Explained\n6. Next Gala\n7. Road-To-Success Workshop\n8. Fast & Creative Gang\n9. Town Hall Meeting\n10. Fierce, Inc."
5,Are there any projects about teaching people new things?,N/A - Cypher not generated,"Yes, there is a project called ""Mentee To Mentor"" that focuses on teaching people new things through content creation strategies."
6,Find projects related to improving business capital.,N/A - Cypher not generated,"Based on the available project data, there is no specific project solely dedicated to improving business capital. However, projects such as ""Leadership Insights"" and ""Active Achievement"" can indirectly help improve business capital through content marketing solutions and the use of AI technology to enhance organizational content. While these projects may contribute to business growth and success, there is no direct information on projects exclusively focused on improving business capital."
7,Tell me about projects involving software solutions for marketing.,N/A - Cypher not generated,"Here are some projects involving software solutions for marketing:\n\n1. **A Triumph Of Softwares**\n - Description: A fully managed content marketing software deal for SaaS companies focusing on customers and big ideas.\n - Complexity: High\n\n2. **Charged By Codes**\n - Description: Marketing solution software that helps founders find their audience and reach influencers to launch their brand.\n - Complexity: Low\n\n3. **Java Dalia**\n - Description: A Software-as-a-Service startup using proprietary AI to generate content, increasing private company content marketing efficiency by 30-60% and generating 80-100% ROI on marketing spend.\n - Complexity: Low\n\n4. **15 Five**\n - Description: A solution for founders to excel at content marketing, utilizing proprietary AI to analyze competition and create optimized content faster.\n - Complexity: Medium\n\nThese projects focus on different aspects of marketing, such as content creation, audience targeting, and competitor analysis, using software solutions to enhance marketing strategies and achieve business goals."
8,What is the total cost of all projects that are about improving processes?,MATCH (p:Project)-[:HAS_TYPE]->(pt:ProjectType {name: 'Improving Processes'})\nRETURN sum(p.cost) AS totalCost,The total cost of all projects that are about improving processes is $0. This indicates that there ar

## Creating Gemini Agent

In [79]:


llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
agent = create_openai_tools_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
gemini_result = test_questions_with_cypher_extraction(agent_executor, is_openai = False)
gemini_result

Starting batch processing of test questions...


Processing Questions:   0%|          | 0/13 [00:00<?, ?it/s]

Processing Questions: 100%|██████████| 13/13 [00:44<00:00,  3.42s/it]


Batch processing complete.


,Question,GEMINI Generated Cypher,GEMINI Final Answer
0,What is the project cost and benefit for the 'Rhinestone' project?,"MATCH (p:Project {name: 'Rhinestone'})\nRETURN p.cost AS projectCost, p.benefit AS projectBenefit","The project cost for the 'Rhinestone' project is $3,648,615.00, and the project benefit is $8,443,980.00."
1,How many projects are currently 'In - Progress'?,MATCH (p:Project)-[:HAS_STATUS]->(s:Status {name: 'In - Progress'})\nRETURN count(p) AS numberOfProjects,There are currently 25 projects that are 'In - Progress'.
2,What is the total benefit of all projects managed by Brenda Chandler?,MATCH (p:Project)-[:MANAGED_BY]->(m:ProjectManager {name: 'Brenda Chandler'})\nRETURN sum(p.benefit) AS totalBenefit,"The total benefit of all projects managed by Brenda Chandler is $131,698,142."
3,Which department is the 'The Blue Bird' project assigned to?,MATCH (p:Project {name: 'The Blue Bird'})-[:ASSIGNED_TO]->(d:Department)\nRETURN d.name AS departmentName,The 'The Blue Bird' project is assigned to the Warehouse department.
4,List the names of all 'Completed' projects in the 'North' region.,"MATCH (p:Project)-[:HAS_STATUS]->(s:Status {name: 'Completed'}),\n (p)-[:LOCATED_IN]->(r:Region {name: 'North'})\nRETURN p.name AS projectName","Based on the available data, I cannot provide the names of all 'Completed' projects in the 'North' region. The query results do not contain this information."
5,Are there any projects about teaching people new things?,N/A - Cypher not generated,"Yes, the ""Mentee To Mentor"" project is about teaching people new things. It focuses on helping individuals develop their skills and knowledge to effectively teach others, particularly in the context of building a communications strategy and creating consistent brand content."
6,Find projects related to improving business capital.,N/A - Cypher not generated,"Based on the provided project data, there is no specific project explicitly mentioned that focuses on improving business capital. The projects mentioned focus more on content marketing, software as a service, and generating better topics and content for organizations. If there are any projects related to improving business capital within the company, they are not detailed in the given context."
7,Tell me about projects involving software solutions for marketing.,N/A - Cypher not generated,"There are three projects that involve software solutions for marketing:\n\n1. **A Triumph Of Softwares**: This is a high-complexity project offering a fully managed content marketing software solution for SaaS companies, focusing on customers and big ideas.\n2. **Charged By Codes**: A low-complexity project, this is a smart marketing software solution designed to help founders find their audience and connect with influencers to launch their brand.\n3. **Sirius**: This high-complexity project provides a customizable, on-demand solution that combines the latest content marketing technology with modern tools for entrepreneurs.\n\nAll three projects aim to enhance marketing strategies, target specific audiences, and optimize content for improved engagement and reach."
8,What is the total cost of all projects that are about improving processes?,MATCH (p:Project)-[:HAS_TYPE]->(pt:ProjectType {name: 'Improving Processes'})\nRETURN sum(p.cost) AS totalCost,The total cost of all projects that are about improving processes is $0.
9,Who is the project manager for the project described as a 'casual game'?,MATCH (p:Project {description: 'casual game'})-[:MANAGED_BY]->(m:ProjectManager)\nRETURN m.name AS projectManagerName,I could not find a project manager for a project described as a 'casual game'.


In [80]:
df_combined = pd.merge(openai_result, gemini_result)

In [81]:
df_combined

,Question,OPENAI Generated Cypher,OPENAI Final Answer,GEMINI Generated Cypher,GEMINI Final Answer
0,What is the project cost and benefit for the 'Rhinestone' project?,"MATCH (p:Project {name: 'Rhinestone'})\nRETURN p.cost AS projectCost, p.benefit AS projectBenefit","The project cost for the 'Rhinestone' project is $3,648,615.00, and the project benefit is $8,443,980.00.","MATCH (p:Project {name: 'Rhinestone'})\nRETURN p.cost AS projectCost, p.benefit AS projectBenefit","The project cost for the 'Rhinestone' project is $3,648,615.00, and the project benefit is $8,443,980.00."
1,How many projects are currently 'In - Progress'?,MATCH (p:Project)-[:HAS_STATUS]->(s:Status {name: 'In - Progress'})\nRETURN count(p) AS numberOfProjects,There are currently 25 projects that are 'In - Progress'.,MATCH (p:Project)-[:HAS_STATUS]->(s:Status {name: 'In - Progress'})\nRETURN count(p) AS numberOfProjects,There are currently 25 projects that are 'In - Progress'.
2,What is the total benefit of all projects managed by Brenda Chandler?,MATCH (p:Project)-[:MANAGED_BY]->(m:ProjectManager {name: 'Brenda Chandler'})\nRETURN sum(p.benefit) AS totalBenefit,"The total benefit of all projects managed by Brenda Chandler is $131,698,142.",MATCH (p:Project)-[:MANAGED_BY]->(m:ProjectManager {name: 'Brenda Chandler'})\nRETURN sum(p.benefit) AS totalBenefit,"The total benefit of all projects managed by Brenda Chandler is $131,698,142."
3,Which department is the 'The Blue Bird' project assigned to?,MATCH (p:Project {name: 'The Blue Bird'})-[:ASSIGNED_TO]->(d:Department)\nRETURN d.name AS departmentName,The 'The Blue Bird' project is assigned to the Warehouse department.,MATCH (p:Project {name: 'The Blue Bird'})-[:ASSIGNED_TO]->(d:Department)\nRETURN d.name AS departmentName,The 'The Blue Bird' project is assigned to the Warehouse department.
4,List the names of all 'Completed' projects in the 'North' region.,"MATCH (p:Project)-[:HAS_STATUS]->(s:Status {name: 'Completed'}),\n (p)-[:LOCATED_IN]->(r:Region {name: 'North'})\nRETURN p.name AS projectName, p.description AS projectDescription, p.startDate AS projectStartDate, p.endDate AS projectEndDate, p.cost AS projectCost, p.benefit AS projectBenefit, p.completionPercentage AS projectCompletionPercentage","The completed projects in the North region are:\n1. Impact Training\n2. Gob Geeklords\n3. Leadership Insights\n4. Commission Kings\n5. Project Explained\n6. Next Gala\n7. Road-To-Success Workshop\n8. Fast & Creative Gang\n9. Town Hall Meeting\n10. Fierce, Inc.","MATCH (p:Project)-[:HAS_STATUS]->(s:Status {name: 'Completed'}),\n (p)-[:LOCATED_IN]->(r:Region {name: 'North'})\nRETURN p.name AS projectName","Based on the available data, I cannot provide the names of all 'Completed' projects in the 'North' region. The query results do not contain this information."
5,Are there any projects about teaching people new things?,N/A - Cypher not generated,"Yes, there is a project called ""Mentee To Mentor"" that focuses on teaching people new things through content creation strategies.",N/A - Cypher not generated,"Yes, the ""Mentee To Mentor"" project is about teaching people new things. It focuses on helping individuals develop their skills and knowledge to effectively teach others, particularly in the context of building a communications strategy and creating consistent brand content."
6,Find projects related to improving business capital.,N/A - Cypher not generated,"Based on the available project data, there is no specific project solely dedicated to improving business capital. However, projects such as ""Leadership Insights"" and ""Active Achievement"" can indirectly help improve business capital through content marketing solutions and the use of AI technology to enhance organizational content. While these projects may contribute to business growth and success, there is no direct information on projects exclusively focused on improving business capital.",N/A - Cypher not generated,"Based on the provided project data, t